In [4]:
import pandas as pd
import numpy as np
import ast


# increasing display to view large descriptions and reviewText
pd.set_option('display.max_colwidth', None)


import warnings
warnings.filterwarnings('ignore')

In [5]:
df_collab = pd.read_csv('./data/reviews.csv')
df_collab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8765568 entries, 0 to 8765567
Data columns (total 12 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   overall         int64 
 1   verified        bool  
 2   reviewTime      object
 3   reviewerID      object
 4   asin            object
 5   style           object
 6   reviewerName    object
 7   reviewText      object
 8   summary         object
 9   unixReviewTime  int64 
 10  vote            object
 11  image           object
dtypes: bool(1), int64(2), object(9)
memory usage: 744.0+ MB


In [6]:
meta_df = pd.read_csv('./data/reviews_meta.csv')
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203766 entries, 0 to 203765
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   category         203766 non-null  object 
 1   tech1            6 non-null       object 
 2   description      203766 non-null  object 
 3   fit              0 non-null       float64
 4   title            203707 non-null  object 
 5   also_buy         203766 non-null  object 
 6   tech2            0 non-null       float64
 7   brand            137335 non-null  object 
 8   feature          203766 non-null  object 
 9   rank             203766 non-null  object 
 10  also_view        203766 non-null  object 
 11  main_cat         203756 non-null  object 
 12  similar_item     0 non-null       float64
 13  date             38 non-null      object 
 14  price            110745 non-null  object 
 15  asin             203766 non-null  object 
 16  imageURL         203766 non-null  obje

In [190]:
# df_copy_collab = df_collab.copy()
# df_copy_meta = meta_df.copy()
##### do not un comment ######

df_collab = df_copy_collab.copy()
meta_df = df_copy_meta.copy()

In [191]:
col_meta = meta_df.copy()
col_meta.rename(columns={'category': 'genre', 'brand': 'starring', 'asin': 'movie_id'}, inplace=True)
col_meta.drop_duplicates(subset='movie_id', inplace=True)
col_meta = col_meta[col_meta['main_cat'] == 'Movies & TV']
col_meta.drop(['tech1', 'fit', 'tech2', 'similar_item',
         'date', 'price', 'imageURL', 'imageURLHighRes',
        'also_buy', 'also_view', 'feature', 'rank',
          'main_cat', 'details'], axis=1, inplace=True)

col_meta.info()

<class 'pandas.core.frame.DataFrame'>
Index: 181552 entries, 0 to 203765
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   genre        181552 non-null  object
 1   description  181552 non-null  object
 2   title        181494 non-null  object
 3   starring     120607 non-null  object
 4   movie_id     181552 non-null  object
dtypes: object(5)
memory usage: 8.3+ MB


In [192]:
df_collab.drop(['image', 'reviewTime', 'reviewerName', 'summary',
              'vote', 'unixReviewTime'], axis=1, inplace=True)

df_collab.rename(columns={'overall': 'rating', 'asin': 'movie_id',
                          'reviewerID': 'user_id','reviewText':'reviews'}, inplace=True)
df_collab.head()

,rating,verified,user_id,movie_id,style,reviews
0,5,True,A3478QRKQDOPQ2,0001527665,{'Format:': ' VHS Tape'},"really happy they got evangelised .. spoiler alert==happy ending liked that..since started bit worrisome... but yeah great stories these missionary movies, really short only half hour but still great"
1,5,True,A2VHSG6TZHU1OB,0001527665,{'Format:': ' Amazon Video'},"Having lived in West New Guinea (Papua) during the time period covered in this video, it is realistic, accurate, and conveys well the entrance of light and truth into a culture that was for centuries dead to and alienated from God."
2,5,False,A23EJWOW1TLENE,0001527665,{'Format:': ' Amazon Video'},Excellent look into contextualizing the Gospel and God's sovereignty over cultural barriers. The book and movie are both captivating. I would definitely recommend to both Christians and non-believers.
3,5,True,A1KM9FNEJ8Q171,0001527665,{'Format:': ' Amazon Video'},"More than anything, I've been challenged to find ways to share Christ is a culturally relevant way to those around me. Peace child is a cherished ""how to"" for me to do that."
4,4,True,A38LY2SSHVHRYB,0001527665,{'Format:': ' Amazon Video'},"This is a great movie for a missionary going into a foreign country, especially one that is not used to foreign presence. But, it was a little on the short side."


In [193]:
rm_format = df_collab['style'].apply(lambda x: isinstance(x, str) and "{'Format:" in x)
df_collab = df_collab.loc[rm_format]

df_collab['style'] = df_collab['style'].apply(lambda x: ast.literal_eval(x))
df_collab['style'] = df_collab['style'].apply(lambda x: x['Format:'])
df_collab['style'] = df_collab['style'].astype(str)

df_collab = df_collab[df_collab['style'].isin(df_collab['style'].value_counts()[df_collab['style'].value_counts() >= 200000].index)]

In [194]:
df_collab = df_collab[df_collab['verified'] == True]
df_collab.drop(columns = ['verified', 'style'], inplace=True)

In [195]:
df_collab.dropna(subset=['reviews'], inplace=True)
df_collab.drop_duplicates(subset=['user_id', 'movie_id'], keep='first', inplace=True)
df_collab.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6212819 entries, 1 to 8765561
Data columns (total 4 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   rating    int64 
 1   user_id   object
 2   movie_id  object
 3   reviews   object
dtypes: int64(1), object(3)
memory usage: 237.0+ MB


In [196]:
# df_copy_collab2 = df_collab.copy()

##### do not un comment ######

df_collab = df_copy_collab2.copy()
df_collab.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6212819 entries, 1 to 8765561
Data columns (total 4 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   rating    int64 
 1   user_id   object
 2   movie_id  object
 3   reviews   object
dtypes: int64(1), object(3)
memory usage: 237.0+ MB


In [197]:
all_vid = df_collab['movie_id'].unique().tolist()
col_meta = col_meta[col_meta['movie_id'].isin(all_vid)]

In [198]:
all_vid2 = col_meta['movie_id'].unique().tolist()
df_collab = df_collab[df_collab['movie_id'].isin(all_vid2)]
df_collab = df_collab[df_collab['user_id'].isin(df_collab['user_id'].value_counts()[df_collab['user_id'].value_counts() >= 5].index)]

In [199]:
all_vid3 = df_collab['movie_id'].unique().tolist()
col_meta = col_meta[col_meta['movie_id'].isin(all_vid3)]

In [200]:
col_meta.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97209 entries, 3 to 203763
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   genre        97209 non-null  object
 1   description  97209 non-null  object
 2   title        97194 non-null  object
 3   starring     76246 non-null  object
 4   movie_id     97209 non-null  object
dtypes: object(5)
memory usage: 4.4+ MB


In [201]:
df_collab.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2163352 entries, 21 to 8765560
Data columns (total 4 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   rating    int64 
 1   user_id   object
 2   movie_id  object
 3   reviews   object
dtypes: int64(1), object(3)
memory usage: 82.5+ MB


In [33]:
col_meta['genre'].value_counts().head(55)

genre
['Movies & TV', 'Movies']                                                                                     8254
['Movies & TV', 'Genre for Featured Categories', 'Action & Adventure']                                        5485
['Movies & TV', 'Genre for Featured Categories', 'Drama']                                                     5155
['Movies & TV', 'Genre for Featured Categories', 'Kids & Family']                                             4004
['Movies & TV', 'Genre for Featured Categories', 'Comedy']                                                    3961
['Movies & TV', 'Genre for Featured Categories', 'Documentary']                                               3642
['Movies & TV', 'Independently Distributed', 'Documentary']                                                   3172
['Movies & TV', 'Studio Specials', 'Sony Pictures Home Entertainment', 'All Sony Pictures Titles']            2964
['Movies & TV', 'Studio Specials', 'Warner Home Video', 'All Titles']     

In [202]:
crub_nub = col_meta.copy()
crub_nub.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97209 entries, 3 to 203763
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   genre        97209 non-null  object
 1   description  97209 non-null  object
 2   title        97194 non-null  object
 3   starring     76246 non-null  object
 4   movie_id     97209 non-null  object
dtypes: object(5)
memory usage: 4.4+ MB


In [203]:
crub_nub['genre'] = crub_nub['genre'].apply(lambda x: ast.literal_eval(x))
crub_nub['genre'] = [x[1:] if len(x) > 1 and x[0] == 'Movies & TV' else x for x in crub_nub['genre']]
crub_nub = crub_nub[~crub_nub['genre'].apply(lambda x: 'Exercise & Fitness' in x)]
crub_nub.loc[crub_nub['genre'].apply(lambda x: isinstance(x, list) and len(x) > 2 and x[0] == 'Art House & International'), 'genre'] = crub_nub['genre'].apply(lambda x: [x[0] + ' ' + x[2]] if len(x) > 2 else x)
crub_nub['genre'] = crub_nub['genre'].apply(lambda x: x[:1] + x[2:] if isinstance(x, list) and len(x) > 2 and x[0] == 'Art House & International' and len(x) > 2 else x)
crub_nub.info()

<class 'pandas.core.frame.DataFrame'>
Index: 94661 entries, 3 to 203763
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   genre        94661 non-null  object
 1   description  94661 non-null  object
 2   title        94646 non-null  object
 3   starring     74371 non-null  object
 4   movie_id     94661 non-null  object
dtypes: object(5)
memory usage: 6.3+ MB


In [204]:
# crub_cop = crub_nub.copy()
meta_df = crub_cop.copy()
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 94661 entries, 3 to 203763
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   genre        94661 non-null  object
 1   description  94661 non-null  object
 2   title        94646 non-null  object
 3   starring     74371 non-null  object
 4   movie_id     94661 non-null  object
dtypes: object(5)
memory usage: 6.3+ MB


In [205]:
remove_genre_str = ['Genre for Featured Categories', 'Blu-ray Movies',
                    'Studio Specials Universal Studios Home Entertainment All Universal Studios Titles',
                    'Studio Specials Sony Pictures Home Entertainment All Sony Pictures Titles',
                    'Studio Specials Lionsgate Home Entertainment All Lionsgate Titles',
                    'Studio Specials Warner Home Video All Titles', 'Independently Distributed',
                    'Studio Specials 20th Century Fox Home Entertainment All Fox Titles',
                    'A&E Home Video All A&E Titles', 'Studio Specials MGM Home Entertainment All MGM Titles',
                    'MOD CreateSpace Video', 'BBC All BBC Titles', 'Independently Distributed',
                    'Boxed Sets', 'Blu-ray TV', 'Fully Loaded DVDs DTS', 'HBO All HBO Titles',
                    'Studio Specials 20th Century Fox Home Entertainment Action General',
                    'Studio Specials 20th Century Fox Home Entertainment Fox TV General',
                    'Walt Disney Studios Home Entertainment All Disney Titles',
                    'Paramount Home Entertainment', 'Fully Loaded DVDs Special Editions',
                    'Studio Specials Miramax Home Entertainment All Titles', 'TV' 'Television'
                   ] 

In [206]:
meta_df['genre'] = meta_df['genre'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
meta_df['genre'].replace({'': 'Unknown'}, inplace=True)

In [207]:
def remove_substrings(s, word_list):
    for word in word_list:
        s = s.replace(word, '')
    return s

mask = ~meta_df['genre'].isin(remove_genre_str)
meta_df = meta_df[mask]
meta_df['genre'] = meta_df['genre'].apply(remove_substrings, word_list=remove_genre_str).str.strip()
len(meta_df['genre'].value_counts())

553

In [208]:
meta_df['genre'] = meta_df['genre'].str.replace('.*Christmas.*', 'Christmas', regex=True)
len(meta_df['genre'].value_counts())

546

In [209]:
meta_df['genre'] = meta_df['genre'].str.replace('.*Anime.*', 'Anime', regex=True)
len(meta_df['genre'].value_counts())

545

In [210]:
meta_df['genre'] = meta_df['genre'].str.replace('.*Animation.*', 'Animation', regex=True)
len(meta_df['genre'].value_counts())

538

In [211]:
meta_df['genre'] = meta_df['genre'].str.replace('.*Reality TV.*', 'Reality TV', regex=True)
len(meta_df['genre'].value_counts())

537

In [212]:
meta_df['genre'] = meta_df['genre'].str.replace('.*Musicals.*', 'Musicals & Performing Arts', regex=True)
meta_df['genre'] = meta_df['genre'].str.replace('.*Performing Arts.*', 'Musicals & Performing Arts', regex=True)
meta_df['genre'] = meta_df['genre'].str.replace('.*Music Video.*', 'Music Videos & Concerts', regex=True)
len(meta_df['genre'].value_counts())

524

In [213]:
meta_df['genre'] = meta_df['genre'].str.replace('.*Art House.*', 'Art House & International', regex=True)
len(meta_df['genre'].value_counts())

468

In [214]:
meta_df['genre'] = meta_df['genre'].str.replace('.*Science Fiction & Fantasy.*', 'Science Fiction & Fantasy', regex=True)
meta_df['genre'] = meta_df['genre'].str.replace('.*Sci-Fi & Fantasy.*', 'Science Fiction & Fantasy', regex=True)
meta_df['genre'] = meta_df['genre'].str.replace('.*Science Fiction.*', 'Science Fiction & Fantasy', regex=True)
meta_df['genre'] = meta_df['genre'].str.replace('.*Fantasy.*', 'Science Fiction & Fantasy', regex=True)
len(meta_df['genre'].value_counts())

447

In [215]:
meta_df['genre'] = meta_df['genre'].str.replace('Action', 'Action & Adventure', regex=True)
meta_df['genre'] = meta_df['genre'].str.replace('.*Action & Adventure.*', 'Action & Adventure', regex=True)
len(meta_df['genre'].value_counts())

435

In [216]:
meta_df['genre'] = meta_df['genre'].str.replace('.*Christian Movies & TV.*', 'Faith & Spirituality', regex=True)
len(meta_df['genre'].value_counts())

434

In [217]:
meta_df['genre'] = meta_df['genre'].str.replace('.*Mystery.*', 'Mystery & Thrillers', regex=True)
len(meta_df['genre'].value_counts())

432

In [218]:
meta_df['genre'] = meta_df['genre'].str.replace('.*News.*', 'News', regex=True)
len(meta_df['genre'].value_counts())

423

In [219]:
meta_df['genre'] = meta_df['genre'].str.replace('.*John Wayne.*', 'Westerns', regex=True)
len(meta_df['genre'].value_counts())

420

In [220]:
meta_df['genre'] = meta_df['genre'].str.replace('.*Classics.*', 'Classics & Silent Film', regex=True)
len(meta_df['genre'].value_counts())

406

In [221]:
meta_df['genre'].value_counts().head(55)

genre
Drama                                                                                8530
Movies                                                                               8254
Documentary                                                                          7441
Action & Adventure                                                                   7016
Comedy                                                                               6424
Kids & Family                                                                        4415
Musicals & Performing Arts                                                           3693
Special Interests                                                                    3118
Science Fiction & Fantasy                                                            2932
Animation                                                                            2301
Horror                                                                               2272
Art 

In [222]:
not_descriptive = ['Movies','Sony Pictures Home Entertainment','Warner Home Video',
                   'TV','All Lionsgate Titles','MOD CreateSpace Video',
                   'A&E Home Video All A&E Titles','All Fox Titles',
                   'All Universal Studios Titles','MGM Home Entertainment All MGM Titles',
                   'BBC','DTS','HBO','Fox TV','Unknown','Special Editions',
                   'Television','Miramax Home Entertainment','PBS', 'Criterion Collection All'
                  ]

In [223]:
meta_df = meta_df[~meta_df['genre'].isin(not_descriptive)]
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64955 entries, 9 to 203763
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   genre        64955 non-null  object
 1   description  64955 non-null  object
 2   title        64947 non-null  object
 3   starring     54038 non-null  object
 4   movie_id     64955 non-null  object
dtypes: object(5)
memory usage: 3.0+ MB


In [224]:
meta_df.dropna(subset='title', inplace=True)
meta_df['genre'].value_counts().head(29)

genre
Drama                                                                                8529
Documentary                                                                          7441
Action & Adventure                                                                   7016
Comedy                                                                               6424
Kids & Family                                                                        4414
Musicals & Performing Arts                                                           3693
Special Interests                                                                    3118
Science Fiction & Fantasy                                                            2931
Animation                                                                            2301
Horror                                                                               2272
Art House & International                                                            1977
Anim

In [225]:
meta_df = meta_df[meta_df['genre'].isin(meta_df['genre'].value_counts()[meta_df['genre'].value_counts() >= 200].index)]
len(meta_df['genre'].value_counts())

21

In [226]:
col_meta = meta_df.copy()
col_meta.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60076 entries, 22 to 203763
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   genre        60076 non-null  object
 1   description  60076 non-null  object
 2   title        60076 non-null  object
 3   starring     50533 non-null  object
 4   movie_id     60076 non-null  object
dtypes: object(5)
memory usage: 2.8+ MB


In [227]:
all_vid = df_collab['movie_id'].unique().tolist()
col_meta = col_meta[col_meta['movie_id'].isin(all_vid)]

In [228]:
all_vid2 = col_meta['movie_id'].unique().tolist()
df_collab = df_collab[df_collab['movie_id'].isin(all_vid2)]
df_collab = df_collab[df_collab['user_id'].isin(df_collab['user_id'].value_counts()[df_collab['user_id'].value_counts() >= 5].index)]

In [229]:
all_vid3 = df_collab['movie_id'].unique().tolist()
col_meta = col_meta[col_meta['movie_id'].isin(all_vid3)]

In [230]:
df_collab.info()

<class 'pandas.core.frame.DataFrame'>
Index: 601477 entries, 71 to 8765560
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   rating    601477 non-null  int64 
 1   user_id   601477 non-null  object
 2   movie_id  601477 non-null  object
 3   reviews   601477 non-null  object
dtypes: int64(1), object(3)
memory usage: 22.9+ MB


In [236]:
df_collab['user_id'].value_counts().head()

user_id
A1NSDP9YZXLMDX    590
A29TKSIWA3JKF3    408
A2NOZB6VZCTOI4    315
A2HVL790PBWYTU    314
A2B73CL3QSYWLB    288
Name: count, dtype: int64

In [239]:
col_meta.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54995 entries, 22 to 203763
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   genre        54995 non-null  object
 1   description  54995 non-null  object
 2   title        54995 non-null  object
 3   starring     46729 non-null  object
 4   movie_id     54995 non-null  object
dtypes: object(5)
memory usage: 2.5+ MB


In [233]:
df_collab.to_csv('./data/collab_model_revs2_reversed.csv',  encoding='utf-8', index=False)
col_meta.to_csv('./data/collab_meta2_reversed.csv',  encoding='utf-8', index=False)

In [ ]:
meta_df['description'] = meta_df['description'].apply(lambda x: " ".join(ast.literal_eval(x)).strip())
meta_df.loc[meta_df['description'].str.len() < 200, 'description'] = np.nan
meta_df.replace({'description': "###############################################################################################################################################################################################################################################################"}, np.nan, inplace=True)
meta_df.dropna(subset='description', inplace=True)
meta_df.drop_duplicates(subset='description', inplace=True)
meta_df.info()

In [242]:
meta_df.loc[meta_df['description'].str.len() < 200, 'description'] = np.nan
meta_df.dropna(subset='description', inplace=True)
meta_df.drop_duplicates(subset='description', inplace=True)
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47089 entries, 26 to 203763
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   genre        47089 non-null  object
 1   description  47089 non-null  object
 2   title        47089 non-null  object
 3   starring     40477 non-null  object
 4   movie_id     47089 non-null  object
dtypes: object(5)
memory usage: 2.2+ MB
